In [21]:
from sklearn import datasets,metrics
from sklearn.model_selection import train_test_split,GridSearchCV,KFold
from sklearn.metrics import precision_score, recall_score, confusion_matrix, classification_report, accuracy_score, f1_score, mean_absolute_error
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from keras import models, layers
from keras.utils import to_categorical, np_utils
from lightgbm import LGBMClassifier
from keras.models import Sequential # 신경망 모델 생성자함수
from keras.layers import Embedding, LSTM, GRU, Dense, Dropout
from keras.models import load_model # 신경망 모델 파일 불러오기
from keras.optimizers import Adam
from keras.preprocessing import sequence
from keras.callbacks import EarlyStopping # 학습조기종료
from tensorflow.keras.layers import Dense, BatchNormalization, Dropout, LSTM, Activation
from tensorflow.keras import regularizers
from tensorflow.keras import callbacks
from xgboost import plot_importance
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
import pandas as pd
import tensorflow as tf
from keras.preprocessing import sequence
from keras.callbacks import EarlyStopping # 학습조기종료
from tensorflow.keras import callbacks
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers
import numpy
from keras.datasets import imdb
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
import numpy as np
from keras.datasets import mnist
from numpy import argmax

In [22]:
df=pd.read_csv("scaled_MiceImputed_weather.csv")
df = df.drop(['Unnamed: 0'], axis=1)

In [23]:
df.head()

,location,SunshineScaled,EvaporationScaled,WindSpeed9amScaled,WindSpeed3pmScaled,WindGustSpeedScaled,Date,MinTemp,MaxTemp,Rainfall,...,Pressure9am,WindDir9am,Cloud9am,Humidity9am,Temp9am,Pressure3pm,WindDir3pm,Cloud3pm,Humidity3pm,Temp3pm
0,45.0,2.924653,1.739768,0.955511,1.648659,2.001480,0.0,12.6,23.2,0.5,...,1002.8,0.0,0.0,66.0,18.3,998.7,9.0,6.0,51.0,19.3
1,45.0,3.093802,1.979480,1.808289,1.308333,2.014903,1.0,10.6,24.1,0.0,...,1003.0,15.0,3.0,30.0,18.6,1000.8,0.0,2.0,35.0,20.5
2,45.0,3.229732,2.077881,1.410987,1.131402,2.079442,2.0,14.2,25.1,0.0,...,1002.9,14.0,0.0,26.0,22.2,1002.8,0.0,0.0,31.0,22.9
3,45.0,2.734299,1.566018,0.587787,1.335001,1.960095,3.0,12.1,21.1,0.0,...,1005.4,12.0,9.0,66.0,17.0,1005.5,2.0,3.0,70.0,18.3
4,45.0,2.122123,0.825986,1.568616,1.722767,2.014903,4.0,11.9,15.4,0.0,...,1011.6,7.0,8.0,90.0,14.7,1013.2,6.0,9.0,89.0,13.9


In [12]:
y = df['TomorrowRain'].values
X = df.drop(['TomorrowRain'], axis=1)
x=X.values

In [13]:
X_train , X_test , y_train , y_test = train_test_split(x, y,test_size=0.2,random_state=42)

In [18]:
X_train.shape

(274964, 22)

In [32]:
def get_stacking_base_datasets(model, X_train_n, y_train_n, X_test_n, n_folds ):
    # 지정된 n_folds값으로 KFold 생성.
    kf = KFold(n_splits=n_folds, shuffle=False, random_state=None)
    #추후에 메타 모델이 사용할 학습 데이터 반환을 위한 넘파이 배열 초기화 
    train_fold_pred = np.zeros((X_train_n.shape[0] ,1 ))
    test_pred = np.zeros((X_test_n.shape[0],n_folds))
    print(model.__class__.__name__ , ' model 시작 ')
    
    for folder_counter , (train_index, valid_index) in enumerate(kf.split(X_train_n)):
        #입력된 학습 데이터에서 기반 모델이 학습/예측할 폴드 데이터 셋 추출 
        print('\t 폴드 세트: ',folder_counter,' 시작 ')
        X_tr = X_train_n[train_index] 
        y_tr = y_train_n[train_index] 
        X_te = X_train_n[valid_index]  
        
        #폴드 세트 내부에서 다시 만들어진 학습 데이터로 기반 모델의 학습 수행.
        model.fit(X_tr , y_tr)       
        #폴드 세트 내부에서 다시 만들어진 검증 데이터로 기반 모델 예측 후 데이터 저장.
        train_fold_pred[valid_index, :] = model.predict(X_te).reshape(-1,1)
        #입력된 원본 테스트 데이터를 폴드 세트내 학습된 기반 모델에서 예측 후 데이터 저장. 
        test_pred[:, folder_counter] = model.predict(X_test_n)
            
    # 폴드 세트 내에서 원본 테스트 데이터를 예측한 데이터를 평균하여 테스트 데이터로 생성 
    test_pred_mean = np.mean(test_pred, axis=1).reshape(-1,1)    
    
    #train_fold_pred는 최종 메타 모델이 사용하는 학습 데이터, test_pred_mean은 테스트 데이터
    return train_fold_pred , test_pred_mean

In [35]:
rf_clf=RandomForestClassifier(random_state=0)
xgb_clf=XGBClassifier(n_estimators=600,learning_rate=0.2, max_depth=10,use_label_encoder=False)
lgbm_clf = LGBMClassifier(n_estimators=2000, num_leaves=95, subsample=0.8, min_child_samples=100,learning_rate = 0.12,
                          max_depth=128)
cat_clf = CatBoostClassifier(iterations=20,depth=10,learning_rate=1,l2_leaf_reg=3,loss_function='Logloss',verbose=True, random_state=123)
lr_final = LGBMClassifier(n_estimators=2000, num_leaves=95, subsample=0.8, min_child_samples=100,learning_rate = 0.12,
                          max_depth=128)

In [36]:
rf_train,rf_test=get_stacking_base_datasets(rf_clf,X_train,y_train,X_test, 7)
xgb_train, xgb_test = get_stacking_base_datasets(xgb_clf, X_train, y_train, X_test, 7)
lgbm_train, lgbm_test = get_stacking_base_datasets(lgbm_clf, X_train, y_train, X_test,  7)    
cat_train, cat_test = get_stacking_base_datasets(cat_clf, X_train, y_train, X_test, 7)

RandomForestClassifier  model 시작 
	 폴드 세트:  0  시작 
	 폴드 세트:  1  시작 
	 폴드 세트:  2  시작 
	 폴드 세트:  3  시작 
	 폴드 세트:  4  시작 
	 폴드 세트:  5  시작 
	 폴드 세트:  6  시작 
XGBClassifier  model 시작 
	 폴드 세트:  0  시작 
	 폴드 세트:  1  시작 
	 폴드 세트:  2  시작 
	 폴드 세트:  3  시작 
	 폴드 세트:  4  시작 
	 폴드 세트:  5  시작 
	 폴드 세트:  6  시작 
LGBMClassifier  model 시작 
	 폴드 세트:  0  시작 
	 폴드 세트:  1  시작 
	 폴드 세트:  2  시작 
	 폴드 세트:  3  시작 
	 폴드 세트:  4  시작 
	 폴드 세트:  5  시작 
	 폴드 세트:  6  시작 
CatBoostClassifier  model 시작 
	 폴드 세트:  0  시작 
0:	learn: 0.4515757	total: 209ms	remaining: 3.96s
1:	learn: 0.4374054	total: 300ms	remaining: 2.7s
2:	learn: 0.4277795	total: 407ms	remaining: 2.31s
3:	learn: 0.4199604	total: 504ms	remaining: 2.02s
4:	learn: 0.4136794	total: 598ms	remaining: 1.79s
5:	learn: 0.4083837	total: 692ms	remaining: 1.61s
6:	learn: 0.4033857	total: 791ms	remaining: 1.47s
7:	learn: 0.3983347	total: 892ms	remaining: 1.34s
8:	learn: 0.3922559	total: 1.01s	remaining: 1.23s
9:	learn: 0.3880808	total: 1.11s	remaining: 1.11s
10:	learn: 0

In [39]:
Stack_final_X_train = np.concatenate((xgb_train, rf_train, lgbm_train, cat_train), axis=1)
Stack_final_X_test = np.concatenate((xgb_test, rf_test, lgbm_test, cat_test), axis=1)
print('원본 학습 피처 데이터 Shape:',X_train.shape, '원본 테스트 피처 Shape:',X_test.shape)
print('스태킹 학습 피처 데이터 Shape:', Stack_final_X_train.shape,
      '스태킹 테스트 피처 데이터 Shape:',Stack_final_X_test.shape)

원본 학습 피처 데이터 Shape: (274964, 23) 원본 테스트 피처 Shape: (68742, 23)
스태킹 학습 피처 데이터 Shape: (274964, 4) 스태킹 테스트 피처 데이터 Shape: (68742, 4)


In [44]:
lr_final.fit(Stack_final_X_train, y_train)
stack_final = lr_final.predict(Stack_final_X_test)
stack_final = (stack_final > 0.5)
print('최종 메타 모델의 예측 정확도: {0:.4f}'.format(accuracy_score(y_test, stack_final)))

최종 메타 모델의 예측 정확도: 0.9151


In [41]:
import sklearn.metrics as mt # 성능지표를 계산import sklearn.metrics as mt
def get_clf_eval(y_test, pred=None):
    confusion = mt.confusion_matrix( y_test, pred)
    accuracy = mt.accuracy_score(y_test , pred)
    precision = mt.precision_score(y_test , pred)
    recall = mt.recall_score(y_test , pred)
    f1 = mt.f1_score(y_test,pred)

    print('오차 행렬:')
    print(confusion)
    
    print('\n정확도: {0:.4f} \n정밀도: {1:.4f} \n재현율: {2:.4f} \n    F1: {3:.4f}'.format(accuracy, precision, recall, f1))

In [45]:
get_clf_eval(y_test,stack_final)

오차 행렬:
[[51269  3305]
 [ 2528 11640]]

정확도: 0.9151 
정밀도: 0.7789 
재현율: 0.8216 
    F1: 0.7996
